In [1]:
from learningAgent import LearningAlgorithm
from environment import Model
from Qtable import QTable
from policyTable import PolicyTable
from test import Test
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from policyTable import PolicyTable
from play import Tournament
import re
import bimatrix
from fractions import Fraction

In [2]:
# np.random.seed(0)

In [3]:
# Parameters for the game:
total_demand = 400
initial_demands = [total_demand/2, total_demand/2]
discount_factor = 1
low_cost = 57
high_cost = 71 
total_stages = 25

In [4]:
# Parameters for the adversary:
number_actions = 60
number_demands = total_demand + 1

In [5]:
high_cost_player = PolicyTable(number_demands,number_actions,high_cost,total_stages)
high_cost_player = high_cost_player.random_reset()
high_cost_probabilities = [1]
high_cost_players = [high_cost_player]
low_cost_players = []


In [6]:
def training(total_demand, discount_factor, costs, total_stages, adversary_probabilities, adversaries, number_episodes):
    
        game = Model(total_demand, costs, total_stages, adversaries, adversary_probabilities)
        constant = int(number_episodes/49)
        Qtable = QTable(total_demand + 1, number_actions, total_stages , learning_rate = [constant,constant])
        algorithm = LearningAlgorithm(game, Qtable, number_episodes, discount_factor)
        
        episode_counter = 0
        number_rounds = 10
        number_episodes_per_round = int(number_episodes / number_rounds)       
        for round_ in range(number_rounds): 
            algorithm.epsilon_greedy_learning(number_episodes_per_round, episode_counter, number_episodes)      
            episode_counter += number_episodes_per_round
        result = Test(game, Qtable, discount_factor, adversary_probabilities, adversaries)
        payoff, _, _, _, _ = result.total_payoff()
        return Qtable.to_policy_table(costs[0]), payoff

In [7]:
def new_equilibrium(low_cost_players, high_cost_players, discount_factor, costs, total_stages, initial_demands):
    
    tournament = Tournament(low_cost_players, high_cost_players, discount_factor, costs, total_stages, initial_demands)
    A,B = tournament.run_tournament()
    number_rows = len(A)
    number_columns = len(A[0])
    
    with open('game.txt', 'w') as f:
        f.write(str(number_rows) + " " + str(number_columns))
        f.write('\n')
        f.write('\n')
        for row in A:
            for column in row:
                f.write(np.array2string(np.array(int(column))) + " ")
            f.write('\n')
        f.write('\n')
        for row in B:
            for column in row:
                f.write(np.array2string(np.array(int(column))) + " ")
            f.write('\n')
        f.write('\n')
    game = bimatrix.bimatrix("game.txt")
    equilibrium = game.tracing(100)
    low_cost_probs, high_cost_probs, low_cost_support, high_cost_support = recover_probs(equilibrium)
    low_cost_probabilities = return_distribution(len(low_cost_players), low_cost_probs, low_cost_support)
    high_cost_probabilities = return_distribution(len(high_cost_players), high_cost_probs, high_cost_support)
    low_cost_payoff = np.matmul(high_cost_probabilities, np.matmul(B,np.transpose(low_cost_probabilities)))
    high_cost_payoff = np.matmul(high_cost_probabilities, np.matmul(A,np.transpose(low_cost_probabilities)))
    return low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff
    

In [8]:
def recover_probs(test):
    high_cost_probs, low_cost_probs, rest = test.split(")")
    _, high_cost_probs = high_cost_probs.split("(")
    high_cost_probs = [float(Fraction(s)) for s in high_cost_probs.split(',')]
    low_cost_probs = low_cost_probs.split("(")[1]
    low_cost_probs = [float(Fraction(s)) for s in low_cost_probs.split(',')]
    _, high_cost_support, low_cost_support = rest.split('[')
    low_cost_support, _ = low_cost_support.split(']')
    low_cost_support = [int(s) for s in low_cost_support.split(',')]
    high_cost_support, _ = high_cost_support.split(']')
    high_cost_support = [int(s) for s in high_cost_support.split(',')]
    return low_cost_probs, high_cost_probs, low_cost_support, high_cost_support

In [9]:
def return_distribution(number_players, cost_probs, cost_support):
    player_probabilities = [0]* number_players
    for index, support in enumerate(cost_support):
        player_probabilities[support] = cost_probs[support]
    return player_probabilities

In [10]:
low_cost_payoff = 0
high_cost_payoff = 0

In [11]:
number_rounds = 50
number_episodes = 500_000
equilibria = list()

In [12]:
for round_ in range(number_rounds):
    print("Round",round_ + 1, " of ", number_rounds)
    
    print("Train low cost:")
    low_cost_player, payoff = training(total_demand, discount_factor, [low_cost, high_cost], total_stages, high_cost_probabilities, high_cost_players, number_episodes)
    if payoff > low_cost_payoff:
        low_cost_players.append(low_cost_player)
        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = new_equilibrium(low_cost_players, high_cost_players, discount_factor, [low_cost, high_cost], total_stages, initial_demands)
        equilibria.append([low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
    
    print("Train high cost:")
    new_high_cost_player, payoff = training(total_demand, discount_factor, [high_cost, low_cost], total_stages, low_cost_probabilities, low_cost_players, number_episodes)
    if payoff > high_cost_payoff:
        high_cost_players.append(new_high_cost_player)
        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = new_equilibrium(low_cost_players, high_cost_players, discount_factor, [low_cost, high_cost], total_stages, initial_demands)
        equilibria.append([low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
        

Round 1  of  50
Train low cost:
100 times found  (1),(1)
    supports: [0][0]
Train high cost:
100 times found  (1,0),(1)
    supports: [0][0]
Round 2  of  50
Train low cost:
100 times found  (1,0),(1,0)
    supports: [0][0]
Train high cost:
99 times found  (1,0,0),(1,0)
    supports: [0][0]
1 times found  (0,0,1),(0,1)
    supports: [2][1]
Round 3  of  50
Train low cost:
30 times found  (0,0,1),(0,1,0)
    supports: [2][1]
58 times found  (1,0,0),(1,0,0)
    supports: [0][0]
12 times found  (261006/422059,65599/422059,95454/422059),(43101/135604,10169/67802,72165/135604)
    supports: [0, 1, 2][0, 1, 2]
Train high cost:
99 times found  (0,0,0,1),(0,0,1)
    supports: [3][2]
1 times found  (1,0,0,0),(1,0,0)
    supports: [0][0]
Round 4  of  50
Train low cost:
99 times found  (0,0,0,1),(0,0,1,0)
    supports: [3][2]
1 times found  (1,0,0,0),(1,0,0,0)
    supports: [0][0]
Train high cost:
100 times found  (0,0,0,1,0),(0,0,1,0)
    supports: [3][2]
Round 5  of  50
Train low cost:
100 time

25 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
74 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0),(0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [14][2]
1 times found  (0,0,0,0,0,0,0,0,0,53296107/990252878,21058779/495126439,0,0,338698132/495126439,0,217442949/990252878,0,0),(0,72602125/207437823,0,0,83729953/414875646,0,0,0,0,0,0,0,79421611/553167528,0,0,56166771/184389176,0)
    supports: [9, 10, 13, 15][1, 4, 12, 15]
Round 18  of  50
Train low cost:
49 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0),(0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [14][2]
44 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
1 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,991/1238,0,247/1238),(198/443,0,0,0,245/443,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [15, 17][0, 4]
6 times found  (0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0)
    sup

53 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,271605/564713,0,0,114/564713,0,292994/564713,0,0,0,0,0),(0,0,118312/181147,0,0,16152/181147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46683/181147,0)
    supports: [14, 17, 19][2, 5, 23]
44 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
1 times found  (0,0,1303/1434,0,131/1434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,11/27,0,0,0,0,0,0,0,0,16/27,0,0)
    supports: [2, 4][13, 22]
1 times found  (0,0,11891/19530,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31741/195300,0,4961/21700,0,0,0,0,0),(0,0,1313/7590,0,0,0,0,0,0,0,0,0,0,889/3795,0,0,0,0,0,0,0,0,409/690,0,0)
    supports: [2, 17, 19][2, 13, 22]
1 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0)
    supports: [18][14]
Train high cost:
41 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,

69 times found  (0,0,0,0,0,0,625/172021,0,0,0,0,0,0,0,85238/172021,0,0,0,0,86158/172021,0,0,0,0,0,0,0,0,0,0),(0,0,47110/100001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13730/100001,0,0,0,39161/100001,0)
    supports: [6, 14, 19][2, 23, 27]
27 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
1 times found  (0,0,0,0,20789963791/163888472563,0,0,0,0,0,0,0,0,0,0,89827521713/163888472563,0,0,0,26118505407/655553890252,125216937207/655553890252,0,0,0,0,0,30874252811/327776945126,0,0,0),(0,0,0,0,156412669543/519209454261,0,0,0,0,0,0,0,0,23865777502/519209454261,0,0,0,0,0,0,0,242459379448/519209454261,86479266095/519209454261,0,0,0,270063829/14032687953,0,0)
    supports: [4, 15, 19, 20, 26][4, 13, 21, 22, 26]
3 times found  (0,0,404/597,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,193/597,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,317/487,0,0,0,0,170/487,0)
    supports: [2, 19

85 times found  (0,0,0,0,0,0,625/172021,0,0,0,0,0,0,0,85238/172021,0,0,0,0,86158/172021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,47110/100001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13730/100001,0,0,0,39161/100001,0,0,0,0,0,0,0)
    supports: [6, 14, 19][2, 23, 27]
9 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
2 times found  (0,0,1303/1434,0,131/1434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,11/27,0,0,0,0,0,0,0,0,16/27,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [2, 4][13, 22]
1 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [18][14]
3 times found  (0,0,404/597,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,193/597,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,317/487,0,0,0,0,170/4

...

5 times found  (0,0,0,0,0,0,0,0,0,306933/28115705,0,0,0,0,0,4491713/5623141,0,4932004/28115705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,418203/28115705,0,0,0,0,0),(9035243/33315004,0,0,0,7486115/16657502,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5816129/33315004,0,0,0,0,0,0,0,0,0,0,1745701/16657502,0,0,0,0)
    supports: [9, 15, 17, 42][0, 4, 31, 42]
47 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [18][14]
13 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
3 times found  (0,0,939617862232165/29049519749259682,0,0,1119773758802401/29049519749259682,0,0,0,0,3090047065993491/29049519749259682,0,0,0,0,0,0,0,0,0,29703520239

37 times found  (0,0,0,0,0,0,0,0,0,202483/490440,0,0,0,0,0,0,0,0,92473/245220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34337/163480),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,21812/26101,0,0,0,0,0,0,0,0,0,0,0,0,4589/78303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8278/78303,0,0,0,0,0)
    supports: [9, 18, 48][14, 27, 42]
13 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,122/129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7/129),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,79/150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71/150,0,0,0,0,0,0,0,0)
    supports: [18, 48][14, 39]
2 times found  (0,0,0,0,0,0,870157658462699/2075750884209889,0,260175851759815/2075750884209889,0,0,0,0,0,31047636697643/2075750884209889,0,0,0,0,44557399182602/2075750884209889,0,0,413287145209688/2075750884209889,0,0,0,205510255721451/2075750884209889,0,251014937175991/2075750884209889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,5607546667079251/26314627712578730,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2601053870648

20 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,122/129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7/129),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,79/150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71/150,0,0,0,0,0,0,0,0,0)
    supports: [18, 48][14, 39]
7 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    supports: [16][1]
34 times found  (0,0,0,0,0,0,0,0,0,202483/490440,0,0,0,0,0,0,0,0,92473/245220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34337/163480),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,21812/26101,0,0,0,0,0,0,0,0,0,0,0,0,4589/78303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8278/78303,0,0,0,0,0,0)
    supports: [9, 18, 48][14, 27, 42]
26 times found  (0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

1 times found  (0,0,0,0,0,0,0,52818403437399488402/1688232934359749562129,0,0,0,0,0,0,0,503119074398660451133/3939210180172748978301,0,0,223563110081481423844/3939210180172748978301,0,105923200150741349528/1688232934359749562129,0,1476935151531674886596/11817630540518246934903,0,0,0,0,0,1595466952697367018782/11817630540518246934903,0,0,0,0,0,0,0,0,0,0,0,0,357831851730497738737/3939210180172748978301,3703553806101308725591/11817630540518246934903,0,0,0,0,0,75215699604332399698/1313070060057582992767,0),(108503426092535913117349/910834708588085755905605,0,0,0,0,0,0,0,0,0,0,0,0,0,44371484472550699412324/182166941717617151181121,0,23158956055769768318109/910834708588085755905605,0,0,0,0,0,39195833669732017569961/910834708588085755905605,0,0,0,34103766269924394621057/910834708588085755905605,0,0,0,0,0,0,97214680634190050710927/910834708588085755905605,0,0,0,0,0,66740351920990961751037/910834708588085755905605,0,0,0,0,123891169831452470076023/910834708588085755905605,0,196169101750736682679

8 times found  (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,122/129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7/129,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,79/150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71/150,0,0,0,0,0,0,0,0,0,0)
    supports: [18, 48][14, 39]
27 times found  (0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0)
    supports: [10][46]
45 times found  (0,0,0,0,0,0,0,0,0,202483/490440,0,0,0,0,0,0,0,0,92473/245220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34337/163480,0,0),(0,0,0,0,0,0,0,0,0,0,0,0,0,0,21812/26101,0,0,0,0,0,0,0,0,0,0,0,0,4589/78303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8278/78303,0,0,0,0,0,0,0)
    supports: [9, 18, 48][14, 27, 42]
1 times found  (0,0,0,0,0,0,0,0,0,4289851958/30213671385,5507868868/30213671385,0,0,0,0,6695103567/10071223795,0,0,0,0,0,0,264313501/30213671385,0,0,0,0,0,0,0,0,

In [13]:
eqilibrium_low_probs = [equilibrium[0] for equilibrium in equilibria]

In [14]:
for probs in eqilibrium_low_probs:
    print(probs)

[1.0]
[1.0]
[1.0, 0]
[1.0, 0]
[1.0, 0, 0]
[0, 0, 1.0]
[0, 0, 1.0, 0]
[0, 0, 1.0, 0]
[0, 0, 1.0, 0, 0]
[0, 0, 1.0, 0, 0]
[0, 0, 1.0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311475409836066, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.36885245901639346, 0.6311